In [1]:
import os
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
import pprint
from sacred import Experiment, Ingredient

import torch

In [2]:
from src.core.utils.config import *

In [5]:
TEST_CONFIG_PATH = "C:\dev\working\cv-train\data\configs\hubmap\sample_hubmap_config.yaml"

In [6]:
conf = OmegaConf.load(TEST_CONFIG_PATH)
schema = OmegaConf.structured(ExperimentConfig)
OmegaConf.merge(schema, conf)

{'optimizer': {'name': 'Adam', 'params': {'lr': 0.01}}, 'scheduler': {'name': 'CosineAnnealingLR', 'params': {'T_max': 50, 'eta_min': 1e-06}}, 'loss': {'name': 'BCEWithLogitsLoss', 'params': {}}, 'model': {'name': 'unet', 'params': {'model_backbone': 'resnet18', 'in_channels': 3, 'num_classes': 1}}, 'dataset': {'name': 'hubmap', 'params': {'name': 'SimpleDataset', 'num_workers': 4}}, 'trainer': None}

In [7]:
pprint.PrettyPrinter(indent=2).pprint(conf)

{'model': {'name': 'unet', 'params': {'model_backbone': 'resnet18', 'in_channels': 3, 'num_classes': 1}}, 'optimizer': {'name': 'Adam', 'params': {'lr': 0.01}}, 'scheduler': {'name': 'CosineAnnealingLR', 'params': {'T_max': 50, 'eta_min': 1e-06}}, 'loss': {'name': 'BCEWithLogitsLoss', 'params': {}}, 'dataset': {'name': 'hubmap', 'params': {'name': 'SimpleDataset', 'num_workers': 4}}}


In [9]:
model_name = conf.model.get("name", "ResNet18")
print(model_name)

unet


In [15]:
model_config = OmegaConf.to_object(conf.model)
pprint.PrettyPrinter(indent=2).pprint(model_config)

{ 'name': 'unet',
  'params': {'in_channels': 3, 'model_backbone': 'resnet18', 'num_classes': 1}}


In [18]:
config_dict = OmegaConf.to_container(conf)
pprint.PrettyPrinter(indent=2).pprint(config_dict)
conf_converted_back = OmegaConf.create(config_dict)

{ 'dataset': { 'name': 'hubmap',
               'params': {'name': 'SimpleDataset', 'num_workers': 4}},
  'loss': {'name': 'BCEWithLogitsLoss', 'params': {}},
  'model': { 'name': 'unet',
             'params': { 'in_channels': 3,
                         'model_backbone': 'resnet18',
                         'num_classes': 1}},
  'optimizer': {'name': 'Adam', 'params': {'lr': 0.01}},
  'scheduler': { 'name': 'CosineAnnealingLR',
                 'params': {'T_max': 50, 'eta_min': 1e-06}}}


In [13]:
param_dict = model_config.get("params", {})
param_dict = OmegaConf.to_container(param_dict)

In [17]:
optimizer = OptimizerConfig(name='Adam', params={'lr': 0.001, 'weight_decay': 0.0005})
scheduler = SchedulerConfig(name='StepLR', params={'step_size': 30, 'gamma': 0.1})
loss = LossConfig(name='CrossEntropyLoss', params={})
model = ModelConfig(name='ResNet18', params={"pretrained": True})
dataset = DatasetConfig(name='CIFAR10', params={})
# trainer = TrainerConfig(name='Trainer', params={"batch_size": 36, "num_epochs": 100, "num_workers": 4, "device": "cuda"})


In [20]:
conf = OmegaConf.load(TEST_CONFIG_PATH)
schema = OmegaConf.structured(ExperimentConfig)

In [ ]:
conf = OmegaConf.merge(schema, conf)

In [23]:
default_test_experiment=ExperimentConfig(
    optimizer=optimizer,
    scheduler=scheduler,
    loss=loss,
    model=model,
    dataset=dataset,
    trainer=trainer
)

In [24]:
schema = OmegaConf.structured(default_test_experiment)
conf = OmegaConf.load(TEST_CONFIG_PATH)
conf = OmegaConf.merge(schema, conf)

In [22]:
conf = OmegaConf.merge(conf, optimizer)
conf = OmegaConf.merge(conf, scheduler)
conf = OmegaConf.merge(conf, loss)
conf = OmegaConf.merge(conf, model)
conf = OmegaConf.merge(conf, trainer)

ValidationError: Merge error: SchedulerConfig is not a subclass of OptimizerConfig. value: {'name': 'StepLR', 'params': {'step_size': 30, 'gamma': 0.1}}
    full_key: 
    object_type=OptimizerConfig

In [31]:
class TinyModel(torch.nn.Module):

    def __init__(self, **_):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x


In [32]:
model = TinyModel()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [33]:
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(adam_optimizer, step_size=30, gamma=0.1)

In [34]:
import inspect
import torch.optim as optim

optimizers = {}

for name, obj in inspect.getmembers(optim):
    if inspect.isclass(obj):
        args = inspect.signature(obj).parameters.keys()
        optimizers[name] = list(args)

print(optimizers)

{'ASGD': ['params', 'lr', 'lambd', 'alpha', 't0', 'weight_decay', 'foreach', 'maximize', 'differentiable'], 'Adadelta': ['params', 'lr', 'rho', 'eps', 'weight_decay', 'foreach', 'maximize', 'differentiable'], 'Adagrad': ['params', 'lr', 'lr_decay', 'weight_decay', 'initial_accumulator_value', 'eps', 'foreach', 'maximize', 'differentiable'], 'Adam': ['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad', 'foreach', 'maximize', 'capturable', 'differentiable', 'fused'], 'AdamW': ['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad', 'maximize', 'foreach', 'capturable', 'differentiable', 'fused'], 'Adamax': ['params', 'lr', 'betas', 'eps', 'weight_decay', 'foreach', 'maximize', 'differentiable'], 'LBFGS': ['params', 'lr', 'max_iter', 'max_eval', 'tolerance_grad', 'tolerance_change', 'history_size', 'line_search_fn'], 'NAdam': ['params', 'lr', 'betas', 'eps', 'weight_decay', 'momentum_decay', 'foreach', 'differentiable'], 'Optimizer': ['params', 'defaults'], 'RAdam': ['params', 'lr